In [1]:
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import time

import pandas as pd 

In [55]:
reviews = pd.read_csv("../datasets/google_local/reviews.csv")
display(reviews.info())
display(reviews.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11453845 entries, 0 to 11453844
Data columns (total 4 columns):
gPlusPlaceId      object
gPlusUserId       object
rating            float64
unixReviewTime    object
dtypes: float64(1), object(3)
memory usage: 349.5+ MB


None

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime
0,108103314380004200232,100000010817154263736,3.0,1372686659
1,102194128241608748649,100000013500285534661,5.0,1342870724
2,101409858828175402384,100000021336848867366,5.0,1390653513
3,101477177500158511502,100000021336848867366,5.0,1389187706
4,106994170641063333085,100000021336848867366,4.0,1390486279


In [52]:
place_ids = list(reviews['gPlusPlaceId'].unique())
user_ids = list(reviews['gPlusUserId'].unique())

place_ids.sort()
user_ids.sort()

print(len(place_ids), place_ids[:3])
print(len(user_ids), user_ids[:3])

3116785 ['100000007848517518160', '100000027961183785551', '100000028989488322699']
5054567 ['100000010817154263736', '100000013500285534661', '100000021336848867366']


In [56]:
start = time.time()

from bisect import bisect_left 
  
def BinarySearch(a, x): 
    i = bisect_left(a, x) 
    if i != len(a) and a[i] == x: 
        return i 
    else: 
        return -1

for i in reviews.index:
    if i%100000==0:
        print(time.time()-start)
        start = time.time()
        print(i)
    pid = reviews.at[i,'gPlusPlaceId']
    uid = reviews.at[i,'gPlusUserId']

    reviews.at[i,'gPlusPlaceId'] = BinarySearch(place_ids,pid)
    reviews.at[i,'gPlusUserId'] = BinarySearch(user_ids,uid)


0.2108461856842041
0
5.007460594177246
100000
4.743478059768677
200000
4.939120531082153
300000
4.669917583465576
400000
5.119900226593018
500000
4.73084020614624
600000
4.80470085144043
700000
4.740260362625122
800000
4.993489503860474
900000
4.868492603302002
1000000
4.786747217178345
1100000
4.9740073680877686
1200000
4.781872272491455
1300000
5.226130247116089
1400000
4.79388427734375
1500000
4.857175827026367
1600000
4.732042551040649
1700000
4.788888931274414
1800000
4.728853464126587
1900000
4.73508620262146
2000000
4.781548023223877
2100000
4.7074689865112305
2200000
5.060934782028198
2300000
4.9958274364471436
2400000
5.124858140945435
2500000
4.90395712852478
2600000
5.271379709243774
2700000
5.298417568206787
2800000
5.109431505203247
2900000
4.716844081878662
3000000
4.904526472091675
3100000
4.850224494934082
3200000
5.091361045837402
3300000
4.997869491577148
3400000
4.9076759815216064
3500000
4.718376636505127
3600000
4.892521858215332
3700000
5.285653591156006
3800000
5

In [57]:
reviews[:10]

,gPlusPlaceId,gPlusUserId,rating,unixReviewTime
0,1368311,0,3.0,1372686659
1,370282,1,5.0,1342870724
2,237940,2,5.0,1390653513
3,249417,2,5.0,1389187706
4,1181533,2,4.0,1390486279
5,1257074,2,5.0,1390192729
6,1350634,2,5.0,1390825528
7,1919338,2,4.0,1390445561
8,2018069,2,4.0,1390387529
9,2348090,2,5.0,1392295463


In [60]:
fname = "../datasets/google_local/reviews_reformatted.csv"

print(reviews[:10])

reviews.to_csv(fname, encoding='utf-8', index=False)

  gPlusPlaceId gPlusUserId  rating unixReviewTime
0      1368311           0     3.0     1372686659
1       370282           1     5.0     1342870724
2       237940           2     5.0     1390653513
3       249417           2     5.0     1389187706
4      1181533           2     4.0     1390486279
5      1257074           2     5.0     1390192729
6      1350634           2     5.0     1390825528
7      1919338           2     4.0     1390445561
8      2018069           2     4.0     1390387529
9      2348090           2     5.0     1392295463


In [58]:
fname = "../datasets/google_local/user_reformatted.csv"

users = pd.Series(user_ids)

print(users[:10])

users.to_csv(fname, encoding='utf-8', index=True)

0    100000010817154263736
1    100000013500285534661
2    100000021336848867366
3    100000030557048145331
4    100000032416892623125
5    100000035085750632094
6    100000036174088924566
7    100000037905537879319
8    100000039815955317902
9    100000042779388982190
dtype: object


In [59]:
fname = "../datasets/google_local/place_reformatted.csv"

places = pd.Series(place_ids)

print(places[:10])

places.to_csv(fname, encoding='utf-8', index=True)

0    100000007848517518160
1    100000027961183785551
2    100000028989488322699
3    100000030371226927960
4    100000031133105350522
5    100000038035038453021
6    100000061163388376076
7    100000061963763467148
8    100000075541128264108
9    100000077955167136431
dtype: object
